In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [ ]:
%aimport skorchsaver

In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import torch

In [7]:
import torchvision.datasets

In [8]:
torch.cuda.is_available()

True

In [9]:
!mkdir -p models

In [10]:
net = skorch.NeuralNetClassifier(
    module=model.Offnet(
        "fashion-mnist",
        input_size=32,
        in_channels=1,
        classes=10,
        total_block_number=24,
        widen_factor=10,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=20,
    max_epochs=10,
    optimizer=lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True),
    callbacks=[
        ('learning_rate_decay',
         skorch.callbacks.LRScheduler(
             policy=torch.optim.lr_scheduler.MultiStepLR,
             monitor=None,
             gamma=0.1,
             milestones=(150, 225)
         )
        ),
        ('progress_bar', skorch.callbacks.ProgressBar()),
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/{net.module.name}.model.pt", monitor=None)),
        ('optimizer_saver', skorchsaver.OptimizerSaver(target="models/{net.module.name}.optimizer.pt")),
        ('history_saver', skorchsaver.HistorySaver(target="models/{net.module.name}.history.json")),
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    lr=0.1,
    train_split=skorch.dataset.CVSplit(5, stratified=False),
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Offnet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): OffBlockGroup(
        (sequential): Sequential(
          (0): OffBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(144, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (dropout): Dropout(p=0.5)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(160, 18, kernel_size=(1, 1), stride=(1, 1))
           

In [11]:
import RandomErasing.transforms

In [12]:
train_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        RandomErasing.transforms.RandomErasing(mean=[0.4914]),
    ])
)

In [ ]:
net.fit(train_dataset, None, epochs=300)

In [ ]:
# net.fit_loop(train_dataset, None, epochs=300)

In [ ]:
test_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=False,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Pad(2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
    ])
)

In [ ]:
net.save_history('models/{net.module.name}.history.json'.format(net=net))